In [ ]:
!git clone https://github.com/JATAYU000/02Hero

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
words = open("/kaggle/working/02Hero/names.txt",'r').read().splitlines()

In [ ]:
print(f"min: {min(len(p) for p in words)} max: {max(len(p) for p in words)} Total: {len(words)}")

In [ ]:
b = {}
for word in words:
    wl = ['<'] + list(word) + ['>']
    for c1,c2 in zip(wl,wl[1:]):
        bigram = (c1,c2)
        b[bigram] = b.get(bigram,0) + 1

In [ ]:
sorted_bigrams = sorted(b.items(),key = lambda kv: kv[1])

In [ ]:
lookup = sorted(list(set(''.join(words))))
str2int = {s:i+1 for i,s in enumerate(lookup)}
str2int['.'] = 0
int2str = {i:s for s,i in str2int.items()}

In [ ]:
import torch

N = torch.zeros((27,27),dtype = torch.int32)

In [ ]:
for word in words:
    wl = ['.'] + list(word) + ['.']
    for c1,c2 in zip(wl,wl[1:]):
        ic1 = str2int[c1]
        ic2 = str2int[c2]
        N[ic1,ic2] += 1

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = int2str[i] + int2str[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

In [ ]:
N[0]

In [ ]:
p = N[0].float()
p = p/p.sum()
p

In [ ]:
g = torch.Generator().manual_seed(2147483647)
torch.multinomial(p,num_samples = 1,replacement = True).item()

In [ ]:
P = (N+1).float()
P /= P.sum(1,keepdim = True)

In [ ]:
for _ in range(10):
    word = '.'
    cur = '.'
    while True:
        p = P[str2int[cur]]
        nxt = torch.multinomial(p,num_samples = 1,replacement = True).item()
        cur = int2str[nxt]
        word += cur
        if cur == '.':
            break
    print(word[1:-1])

In [ ]:
log_like = 0.0
n = 0
for word in ['shrivaths']:
    wl = ['.'] + list(word) + ['.']
    for c1,c2 in zip(wl,wl[1:]):
        ic1 = str2int[c1]
        ic2 = str2int[c2]
        prob = P[ic1,ic2]
        logprob = torch.log(prob)
        log_like += logprob
        n += 1

print(-log_like/n)

In [ ]:
log_like = 0.0
n = 0
for word in ['shrivathsjq']:
    wl = ['.'] + list(word) + ['.']
    for c1,c2 in zip(wl,wl[1:]):
        ic1 = str2int[c1]
        ic2 = str2int[c2]
        prob = P[ic1,ic2]
        logprob = torch.log(prob)
        log_like += logprob
        n += 1

print(-log_like/n)

In [ ]:
## create training set
import torch

xs = []
ys = []
for word in words:
    wl = ['.'] + list(word) + ['.']
    for c1,c2 in zip(wl,wl[1:]):
        ic1 = str2int[c1]
        ic2 = str2int[c2]
        xs.append(ic1)
        ys.append(ic2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
import torch.nn.functional as F
xenc = F.one_hot(xs,num_classes = 27).float()
xenc.dtype

In [ ]:
W = torch.randn((27,27),requires_grad = True)  # 27 neurons in the first layer

In [ ]:
## forward
layer1 = xenc @ W
counts = layer1.exp()
prob = counts / counts.sum(1,keepdim = True)
prob.shape
loss = -prob[torch.arange(228146),ys].log().mean()
loss.item()

In [ ]:
## backward
W.grad = None
loss.backward()

In [ ]:
W.data += -0.1 * W.grad

In [ ]:
for i in range(100):
    layer1 = xenc @ W
    counts = layer1.exp()
    prob = counts / counts.sum(1,keepdim = True)
    prob.shape
    loss = -prob[torch.arange(228146),ys].log().mean() + 0.001 * (W**2).mean()
    # print(loss.item())
    
    W.grad = None
    loss.backward()
    
    W.data += -60 * W.grad

print(loss.item())

In [ ]:
x = [0]
x = torch.tensor(x)
x = F.one_hot(x,num_classes=27).float()

c = (x @ W ).exp()
p = c / c.sum(1,keepdim = True)
p

In [ ]:


for i in range(5):
  
  out = []
  ix = 0
  while True:
    
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True,).item()
    out.append(int2str[ix])
    if ix == 0:
      break
  print(''.join(out))